In [1]:
from pathlib import Path
from malid import config, logger
from malid.train import train_metamodel
from malid.datamodels import (
    GeneLocus,
    TargetObsColumnEnum,
    map_cross_validation_split_strategy_to_default_target_obs_column,
)
import pandas as pd
from IPython.display import display, Markdown
from typing import List

In [2]:
base_model_train_fold_name = "train_smaller"
metamodel_fold_label_train = "validation"

In [3]:
models_of_interest = config.model_names_to_train
models_of_interest

['dummy_most_frequent',
 'dummy_stratified',
 'lasso_cv',
 'elasticnet_cv0.75',
 'elasticnet_cv',
 'elasticnet_cv0.25',
 'ridge_cv',
 'logisticregression_unregularized',
 'rf_multiclass',
 'linearsvm_ovr']

In [4]:
# We only support split strategies with default target obs column == TargetObsColumnEnum.disease
assert (
    map_cross_validation_split_strategy_to_default_target_obs_column[
        config.cross_validation_split_strategy
    ]
    == TargetObsColumnEnum.disease
)

In [5]:
def choose(gene_locus: GeneLocus, classification_targets: List[TargetObsColumnEnum]):
    for target_obs_column in classification_targets:
        try:
            flavors = train_metamodel.get_metamodel_flavors(
                gene_locus=gene_locus,
                target_obs_column=target_obs_column,
                fold_id=config.all_fold_ids[0],
                base_model_train_fold_name=base_model_train_fold_name,
                use_stubs_instead_of_submodels=True,
            )
        except Exception as err:
            logger.warning(
                f"Failed to generate metamodel flavors for {gene_locus}, {target_obs_column}: {err}"
            )
            continue

        for metamodel_flavor, metamodel_config in flavors.items():
            _output_suffix = (
                Path(gene_locus.name)
                / target_obs_column.name
                / metamodel_flavor
                / f"{base_model_train_fold_name}_applied_to_{metamodel_fold_label_train}_model"
            )
            results_output_prefix = (
                config.paths.second_stage_blending_metamodel_output_dir / _output_suffix
            )

            display(
                Markdown(
                    f"# {gene_locus}, {target_obs_column}, metamodel flavor {metamodel_flavor}"
                )
            )

            try:
                ## All results in a table
                df = pd.read_csv(
                    f"{results_output_prefix}.compare_model_scores.test_set_performance.tsv",
                    sep="\t",
                    index_col=0,
                )
                df["missing_classes"] = df["missing_classes"].map(
                    {False: "✅ no", True: "❗ Missing classes!"}
                )
                df = df.loc[df.index.intersection(models_of_interest)].sort_values(
                    "ROC-AUC (weighted OvO) per fold", ascending=False
                )
                if "Accuracy global with abstention" in df.columns:
                    df = df[
                        [
                            "ROC-AUC (weighted OvO) per fold",
                            "au-PRC (weighted OvO) per fold",
                            "Accuracy global with abstention",
                            "MCC global with abstention",
                            "abstention_rate",
                            "sample_size including abstentions",
                            "n_abstentions",
                            "missing_classes",
                        ]
                    ]
                else:
                    df = df[
                        [
                            "ROC-AUC (weighted OvO) per fold",
                            "au-PRC (weighted OvO) per fold",
                            "Accuracy global",
                            "MCC global",
                            "abstention_rate",
                            "sample_size including abstentions",
                            "n_abstentions",
                            "missing_classes",
                        ]
                    ]
                display(df)
            except Exception as err:
                logger.warning(
                    f"{gene_locus}, {target_obs_column} flavor '{metamodel_flavor}': not yet run: {err}"
                )
                continue

In [6]:
for single_gene_locus in config.gene_loci_used:
    choose(single_gene_locus, config.classification_targets)

# GeneLocus.BCR, TargetObsColumnEnum.age_group_pediatric_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
lasso_cv,0.988 +/- 0.021 (in 3 folds),0.945 +/- 0.095 (in 3 folds),0.461,0.293,0.465969,191,89,✅ no
logisticregression_unregularized,0.988 +/- 0.021 (in 3 folds),0.945 +/- 0.095 (in 3 folds),0.450,0.250,0.465969,191,89,✅ no
linearsvm_ovr,0.988 +/- 0.021 (in 3 folds),0.945 +/- 0.095 (in 3 folds),0.450,0.250,0.465969,191,89,✅ no
elasticnet_cv0.75,0.988 +/- 0.021 (in 3 folds),0.945 +/- 0.095 (in 3 folds),0.482,0.312,0.465969,191,89,✅ no
elasticnet_cv0.25,0.988 +/- 0.021 (in 3 folds),0.945 +/- 0.095 (in 3 folds),0.476,0.274,0.465969,191,89,✅ no
elasticnet_cv,0.988 +/- 0.021 (in 3 folds),0.945 +/- 0.095 (in 3 folds),0.482,0.295,0.465969,191,89,✅ no
ridge_cv,0.987 +/- 0.022 (in 3 folds),0.942 +/- 0.100 (in 3 folds),0.476,0.274,0.465969,191,89,✅ no
rf_multiclass,0.979 +/- 0.019 (in 3 folds),0.915 +/- 0.105 (in 3 folds),0.471,0.219,0.465969,191,89,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.251 +/- 0.109 (in 3 folds),0.429,-0.057,0.465969,191,89,❗ Missing classes!
dummy_stratified,0.473 +/- 0.037 (in 3 folds),0.242 +/- 0.098 (in 3 folds),0.351,-0.042,0.465969,191,89,✅ no


# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.969 +/- 0.003 (in 3 folds),0.969 +/- 0.003 (in 3 folds),0.845,0.777,0.011905,420,5,✅ no
ridge_cv,0.968 +/- 0.006 (in 3 folds),0.969 +/- 0.007 (in 3 folds),0.840,0.772,0.011905,420,5,✅ no
elasticnet_cv0.25,0.965 +/- 0.009 (in 3 folds),0.966 +/- 0.009 (in 3 folds),0.843,0.775,0.011905,420,5,✅ no
rf_multiclass,0.964 +/- 0.009 (in 3 folds),0.965 +/- 0.008 (in 3 folds),0.838,0.763,0.011905,420,5,✅ no
elasticnet_cv,0.963 +/- 0.012 (in 3 folds),0.965 +/- 0.011 (in 3 folds),0.838,0.768,0.011905,420,5,✅ no
elasticnet_cv0.75,0.962 +/- 0.013 (in 3 folds),0.964 +/- 0.012 (in 3 folds),0.845,0.778,0.011905,420,5,✅ no
lasso_cv,0.957 +/- 0.009 (in 3 folds),0.958 +/- 0.010 (in 3 folds),0.829,0.753,0.011905,420,5,✅ no
logisticregression_unregularized,0.956 +/- 0.011 (in 3 folds),0.956 +/- 0.012 (in 3 folds),0.805,0.724,0.011905,420,5,✅ no
dummy_stratified,0.504 +/- 0.004 (in 3 folds),0.506 +/- 0.001 (in 3 folds),0.333,0.014,0.011905,420,5,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.450,0.005,0.011905,420,5,❗ Missing classes!


# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor with_demographics_columns

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv0.25,0.688 +/- 0.035 (in 3 folds),0.719 +/- 0.008 (in 3 folds),0.394,0.271,0.0,165,0,✅ no
lasso_cv,0.687 +/- 0.033 (in 3 folds),0.715 +/- 0.013 (in 3 folds),0.424,0.312,0.0,165,0,✅ no
elasticnet_cv,0.685 +/- 0.034 (in 3 folds),0.716 +/- 0.014 (in 3 folds),0.412,0.291,0.0,165,0,✅ no
ridge_cv,0.684 +/- 0.047 (in 3 folds),0.718 +/- 0.024 (in 3 folds),0.412,0.293,0.0,165,0,✅ no
elasticnet_cv0.75,0.680 +/- 0.034 (in 3 folds),0.712 +/- 0.014 (in 3 folds),0.412,0.292,0.0,165,0,✅ no
rf_multiclass,0.667 +/- 0.063 (in 3 folds),0.706 +/- 0.035 (in 3 folds),0.382,0.257,0.0,165,0,❗ Missing classes!
logisticregression_unregularized,0.662 +/- 0.073 (in 3 folds),0.697 +/- 0.052 (in 3 folds),0.345,0.211,0.0,165,0,❗ Missing classes!
linearsvm_ovr,0.657 +/- 0.070 (in 3 folds),0.687 +/- 0.048 (in 3 folds),0.327,0.187,0.0,165,0,❗ Missing classes!
dummy_stratified,0.500 +/- 0.017 (in 3 folds),0.522 +/- 0.014 (in 3 folds),0.170,-0.002,0.0,165,0,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.176,-0.049,0.0,165,0,❗ Missing classes!


# GeneLocus.TCR, TargetObsColumnEnum.lupus_vs_healthy, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
lasso_cv,0.967 +/- 0.021 (in 3 folds),0.918 +/- 0.032 (in 3 folds),0.854,0.666,0.011494,261,3,✅ no
logisticregression_unregularized,0.967 +/- 0.021 (in 3 folds),0.918 +/- 0.032 (in 3 folds),0.851,0.654,0.011494,261,3,✅ no
linearsvm_ovr,0.967 +/- 0.020 (in 3 folds),0.917 +/- 0.030 (in 3 folds),0.847,0.653,0.011494,261,3,✅ no
elasticnet_cv0.75,0.967 +/- 0.020 (in 3 folds),0.919 +/- 0.029 (in 3 folds),0.858,0.678,0.011494,261,3,✅ no
elasticnet_cv,0.966 +/- 0.022 (in 3 folds),0.918 +/- 0.031 (in 3 folds),0.862,0.690,0.011494,261,3,✅ no
elasticnet_cv0.25,0.964 +/- 0.029 (in 3 folds),0.925 +/- 0.036 (in 3 folds),0.858,0.678,0.011494,261,3,✅ no
ridge_cv,0.956 +/- 0.033 (in 3 folds),0.909 +/- 0.041 (in 3 folds),0.854,0.672,0.011494,261,3,✅ no
rf_multiclass,0.920 +/- 0.031 (in 3 folds),0.843 +/- 0.057 (in 3 folds),0.877,0.697,0.011494,261,3,✅ no
dummy_stratified,0.508 +/- 0.038 (in 3 folds),0.250 +/- 0.015 (in 3 folds),0.648,0.019,0.011494,261,3,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.244 +/- 0.003 (in 3 folds),0.747,0.011,0.011494,261,3,❗ Missing classes!


# GeneLocus.TCR, TargetObsColumnEnum.sex_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
dummy_stratified,0.513 +/- 0.032 (in 3 folds),0.570 +/- 0.075 (in 3 folds),0.455,0.048,0.151515,165,25,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.563 +/- 0.087 (in 3 folds),0.442,0.006,0.151515,165,25,✅ no
rf_multiclass,0.471 +/- 0.107 (in 3 folds),0.587 +/- 0.093 (in 3 folds),0.424,-0.001,0.151515,165,25,✅ no
ridge_cv,0.455 +/- 0.077 (in 3 folds),0.537 +/- 0.092 (in 3 folds),0.388,-0.031,0.151515,165,25,✅ no
logisticregression_unregularized,0.393 +/- 0.023 (in 3 folds),0.517 +/- 0.106 (in 3 folds),0.406,-0.033,0.151515,165,25,✅ no
linearsvm_ovr,0.392 +/- 0.024 (in 3 folds),0.517 +/- 0.106 (in 3 folds),0.406,-0.033,0.151515,165,25,✅ no
elasticnet_cv0.25,0.392 +/- 0.024 (in 3 folds),0.516 +/- 0.106 (in 3 folds),0.406,-0.033,0.151515,165,25,✅ no
elasticnet_cv0.75,0.391 +/- 0.026 (in 3 folds),0.524 +/- 0.105 (in 3 folds),0.400,-0.045,0.151515,165,25,✅ no
elasticnet_cv,0.391 +/- 0.026 (in 3 folds),0.516 +/- 0.106 (in 3 folds),0.400,-0.045,0.151515,165,25,✅ no
lasso_cv,0.389 +/- 0.029 (in 3 folds),0.523 +/- 0.105 (in 3 folds),0.376,-0.085,0.151515,165,25,✅ no


In [7]:
if len(config.gene_loci_used) > 1:
    choose(config.gene_loci_used, config.classification_targets)

# GeneLocus.BCR|TCR, TargetObsColumnEnum.age_group_pediatric_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,1.000 +/- 0.001 (in 3 folds),0.995 +/- 0.008 (in 3 folds),0.527,0.351,0.454545,165,75,✅ no
elasticnet_cv0.75,1.000 +/- 0.000 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.533,0.373,0.454545,165,75,✅ no
elasticnet_cv,1.000 +/- 0.000 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.527,0.351,0.454545,165,75,✅ no
ridge_cv,1.000 +/- 0.000 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.545,0.418,0.454545,165,75,✅ no
elasticnet_cv0.25,1.000 +/- 0.000 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.539,0.395,0.454545,165,75,✅ no
linearsvm_ovr,1.000 +/- 0.000 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.545,0.418,0.454545,165,75,✅ no
logisticregression_unregularized,1.000 +/- 0.000 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.545,0.418,0.454545,165,75,✅ no
lasso_cv,0.990 +/- 0.018 (in 3 folds),0.967 +/- 0.058 (in 3 folds),0.521,0.328,0.454545,165,75,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.271 +/- 0.107 (in 3 folds),0.424,-0.057,0.454545,165,75,❗ Missing classes!
dummy_stratified,0.484 +/- 0.092 (in 3 folds),0.282 +/- 0.137 (in 3 folds),0.333,-0.057,0.454545,165,75,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv0.25,0.988 +/- 0.004 (in 3 folds),0.987 +/- 0.005 (in 3 folds),0.872,0.819,0.013966,358,5,✅ no
elasticnet_cv,0.988 +/- 0.003 (in 3 folds),0.988 +/- 0.004 (in 3 folds),0.874,0.825,0.013966,358,5,✅ no
ridge_cv,0.987 +/- 0.004 (in 3 folds),0.987 +/- 0.005 (in 3 folds),0.872,0.820,0.013966,358,5,✅ no
elasticnet_cv0.75,0.987 +/- 0.003 (in 3 folds),0.986 +/- 0.004 (in 3 folds),0.872,0.819,0.013966,358,5,✅ no
rf_multiclass,0.984 +/- 0.007 (in 3 folds),0.984 +/- 0.006 (in 3 folds),0.899,0.855,0.013966,358,5,✅ no
lasso_cv,0.984 +/- 0.006 (in 3 folds),0.981 +/- 0.010 (in 3 folds),0.877,0.825,0.013966,358,5,✅ no
logisticregression_unregularized,0.977 +/- 0.013 (in 3 folds),0.969 +/- 0.018 (in 3 folds),0.872,0.816,0.013966,358,5,✅ no
linearsvm_ovr,0.975 +/- 0.011 (in 3 folds),0.971 +/- 0.015 (in 3 folds),0.866,0.806,0.013966,358,5,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.455,0.006,0.013966,358,5,❗ Missing classes!
dummy_stratified,0.478 +/- 0.031 (in 3 folds),0.498 +/- 0.015 (in 3 folds),0.302,-0.037,0.013966,358,5,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor with_demographics_columns

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.660 +/- 0.012 (in 3 folds),0.714 +/- 0.017 (in 3 folds),0.255,0.162,0.278788,165,46,✅ no
elasticnet_cv0.75,0.652 +/- 0.047 (in 3 folds),0.706 +/- 0.042 (in 3 folds),0.255,0.166,0.278788,165,46,✅ no
elasticnet_cv0.25,0.645 +/- 0.029 (in 3 folds),0.702 +/- 0.028 (in 3 folds),0.236,0.139,0.278788,165,46,✅ no
ridge_cv,0.644 +/- 0.024 (in 3 folds),0.702 +/- 0.027 (in 3 folds),0.261,0.166,0.278788,165,46,❗ Missing classes!
logisticregression_unregularized,0.639 +/- 0.038 (in 3 folds),0.699 +/- 0.037 (in 3 folds),0.242,0.146,0.278788,165,46,✅ no
rf_multiclass,0.634 +/- 0.086 (in 3 folds),0.689 +/- 0.042 (in 3 folds),0.261,0.165,0.278788,165,46,❗ Missing classes!
elasticnet_cv,0.631 +/- 0.020 (in 3 folds),0.692 +/- 0.029 (in 3 folds),0.224,0.124,0.278788,165,46,✅ no
lasso_cv,0.629 +/- 0.038 (in 3 folds),0.695 +/- 0.050 (in 3 folds),0.267,0.181,0.278788,165,46,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.121,-0.015,0.278788,165,46,❗ Missing classes!
dummy_stratified,0.493 +/- 0.024 (in 3 folds),0.517 +/- 0.010 (in 3 folds),0.115,-0.011,0.278788,165,46,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.lupus_vs_healthy, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv0.25,0.980 +/- 0.010 (in 3 folds),0.956 +/- 0.016 (in 3 folds),0.881,0.715,0.030651,261,8,✅ no
lasso_cv,0.979 +/- 0.017 (in 3 folds),0.954 +/- 0.029 (in 3 folds),0.866,0.692,0.030651,261,8,✅ no
elasticnet_cv,0.979 +/- 0.012 (in 3 folds),0.955 +/- 0.019 (in 3 folds),0.862,0.680,0.030651,261,8,✅ no
logisticregression_unregularized,0.979 +/- 0.007 (in 3 folds),0.938 +/- 0.022 (in 3 folds),0.893,0.746,0.030651,261,8,✅ no
elasticnet_cv0.75,0.978 +/- 0.015 (in 3 folds),0.952 +/- 0.027 (in 3 folds),0.866,0.687,0.030651,261,8,✅ no
ridge_cv,0.977 +/- 0.008 (in 3 folds),0.948 +/- 0.014 (in 3 folds),0.870,0.689,0.030651,261,8,✅ no
linearsvm_ovr,0.977 +/- 0.007 (in 3 folds),0.937 +/- 0.019 (in 3 folds),0.870,0.698,0.030651,261,8,✅ no
rf_multiclass,0.968 +/- 0.017 (in 3 folds),0.933 +/- 0.040 (in 3 folds),0.889,0.719,0.030651,261,8,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.245 +/- 0.008 (in 3 folds),0.732,0.001,0.030651,261,8,❗ Missing classes!
dummy_stratified,0.465 +/- 0.065 (in 3 folds),0.243 +/- 0.012 (in 3 folds),0.602,-0.066,0.030651,261,8,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.sex_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
dummy_most_frequent,0.500 +/- 0.000 (in 2 folds),0.559 +/- 0.029 (in 2 folds),0.273,-0.035,0.418182,110,46,✅ no
dummy_stratified,0.489 +/- 0.008 (in 2 folds),0.553 +/- 0.025 (in 2 folds),0.291,0.003,0.418182,110,46,✅ no
rf_multiclass,0.475 +/- 0.140 (in 2 folds),0.606 +/- 0.130 (in 2 folds),0.300,0.014,0.418182,110,46,✅ no
elasticnet_cv0.75,0.462 +/- 0.054 (in 2 folds),0.544 +/- 0.008 (in 2 folds),0.255,-0.073,0.418182,110,46,✅ no
elasticnet_cv,0.460 +/- 0.056 (in 2 folds),0.543 +/- 0.007 (in 2 folds),0.255,-0.073,0.418182,110,46,✅ no
lasso_cv,0.459 +/- 0.058 (in 2 folds),0.542 +/- 0.006 (in 2 folds),0.245,-0.091,0.418182,110,46,✅ no
ridge_cv,0.457 +/- 0.060 (in 2 folds),0.531 +/- 0.011 (in 2 folds),0.264,-0.056,0.418182,110,46,✅ no
logisticregression_unregularized,0.399 +/- 0.034 (in 2 folds),0.522 +/- 0.040 (in 2 folds),0.264,-0.051,0.418182,110,46,✅ no
linearsvm_ovr,0.399 +/- 0.026 (in 2 folds),0.522 +/- 0.036 (in 2 folds),0.264,-0.051,0.418182,110,46,✅ no
elasticnet_cv0.25,0.396 +/- 0.026 (in 2 folds),0.509 +/- 0.020 (in 2 folds),0.245,-0.084,0.418182,110,46,✅ no


In [8]:
# Default
for target in [
    TargetObsColumnEnum.disease,
]:
    for gene_locus in config.gene_loci_used:
        choose(gene_locus, [target])
    choose(config.gene_loci_used, [target])

# GeneLocus.BCR, TargetObsColumnEnum.disease, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
ridge_cv,0.959 +/- 0.001 (in 3 folds),0.959 +/- 0.002 (in 3 folds),0.740,0.676,0.024351,616,15,✅ no
elasticnet_cv0.25,0.958 +/- 0.001 (in 3 folds),0.958 +/- 0.003 (in 3 folds),0.737,0.672,0.024351,616,15,✅ no
elasticnet_cv,0.956 +/- 0.003 (in 3 folds),0.956 +/- 0.001 (in 3 folds),0.734,0.668,0.024351,616,15,✅ no
linearsvm_ovr,0.954 +/- 0.001 (in 3 folds),0.955 +/- 0.001 (in 3 folds),0.750,0.686,0.024351,616,15,✅ no
elasticnet_cv0.75,0.949 +/- 0.009 (in 3 folds),0.952 +/- 0.005 (in 3 folds),0.734,0.668,0.024351,616,15,✅ no
rf_multiclass,0.949 +/- 0.004 (in 3 folds),0.948 +/- 0.008 (in 3 folds),0.737,0.661,0.024351,616,15,✅ no
lasso_cv,0.948 +/- 0.010 (in 3 folds),0.950 +/- 0.006 (in 3 folds),0.726,0.657,0.024351,616,15,✅ no
logisticregression_unregularized,0.943 +/- 0.006 (in 3 folds),0.945 +/- 0.002 (in 3 folds),0.721,0.650,0.024351,616,15,✅ no
dummy_stratified,0.514 +/- 0.018 (in 3 folds),0.511 +/- 0.008 (in 3 folds),0.231,0.027,0.024351,616,15,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.359,0.021,0.024351,616,15,❗ Missing classes!


# GeneLocus.BCR, TargetObsColumnEnum.disease, metamodel flavor subset_of_submodels_repertoire_stats

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv0.75,0.919 +/- 0.008 (in 3 folds),0.923 +/- 0.003 (in 3 folds),0.709,0.634,0.0,616,0,✅ no
ridge_cv,0.919 +/- 0.007 (in 3 folds),0.923 +/- 0.004 (in 3 folds),0.701,0.626,0.0,616,0,✅ no
elasticnet_cv0.25,0.918 +/- 0.008 (in 3 folds),0.923 +/- 0.004 (in 3 folds),0.705,0.630,0.0,616,0,✅ no
elasticnet_cv,0.917 +/- 0.012 (in 3 folds),0.922 +/- 0.005 (in 3 folds),0.713,0.637,0.0,616,0,✅ no
linearsvm_ovr,0.917 +/- 0.010 (in 3 folds),0.922 +/- 0.005 (in 3 folds),0.706,0.625,0.0,616,0,✅ no
lasso_cv,0.916 +/- 0.011 (in 3 folds),0.922 +/- 0.005 (in 3 folds),0.708,0.632,0.0,616,0,✅ no
rf_multiclass,0.907 +/- 0.007 (in 3 folds),0.911 +/- 0.005 (in 3 folds),0.727,0.646,0.0,616,0,✅ no
logisticregression_unregularized,0.900 +/- 0.004 (in 3 folds),0.906 +/- 0.007 (in 3 folds),0.695,0.615,0.0,616,0,✅ no
dummy_stratified,0.512 +/- 0.026 (in 3 folds),0.514 +/- 0.012 (in 3 folds),0.227,0.016,0.0,616,0,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.364,0.000,0.0,616,0,❗ Missing classes!


# GeneLocus.BCR, TargetObsColumnEnum.disease, metamodel flavor subset_of_submodels_convergent_cluster_model

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
ridge_cv,0.982 +/- 0.005 (in 3 folds),0.981 +/- 0.004 (in 3 folds),0.840,0.799,0.0,550,0,✅ no
elasticnet_cv,0.980 +/- 0.005 (in 3 folds),0.980 +/- 0.005 (in 3 folds),0.847,0.806,0.0,550,0,✅ no
elasticnet_cv0.25,0.980 +/- 0.004 (in 3 folds),0.979 +/- 0.003 (in 3 folds),0.842,0.800,0.0,550,0,✅ no
elasticnet_cv0.75,0.978 +/- 0.005 (in 3 folds),0.977 +/- 0.005 (in 3 folds),0.833,0.788,0.0,550,0,✅ no
rf_multiclass,0.977 +/- 0.006 (in 3 folds),0.977 +/- 0.004 (in 3 folds),0.862,0.822,0.0,550,0,✅ no
lasso_cv,0.975 +/- 0.008 (in 3 folds),0.974 +/- 0.008 (in 3 folds),0.831,0.786,0.0,550,0,✅ no
logisticregression_unregularized,0.968 +/- 0.010 (in 3 folds),0.970 +/- 0.007 (in 3 folds),0.822,0.773,0.0,550,0,✅ no
linearsvm_ovr,0.965 +/- 0.007 (in 3 folds),0.969 +/- 0.005 (in 3 folds),0.827,0.780,0.0,550,0,✅ no
dummy_stratified,0.502 +/- 0.013 (in 3 folds),0.511 +/- 0.007 (in 3 folds),0.218,-0.001,0.0,550,0,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.358,0.000,0.0,550,0,❗ Missing classes!


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease, metamodel flavor subset_of_submodels_convergent_cluster_model_sequence_model

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv,0.983 +/- 0.006 (in 3 folds),0.982 +/- 0.006 (in 3 folds),0.838,0.798,0.029091,550,16,✅ no
ridge_cv,0.982 +/- 0.008 (in 3 folds),0.981 +/- 0.008 (in 3 folds),0.807,0.764,0.029091,550,16,✅ no
elasticnet_cv0.25,0.981 +/- 0.006 (in 3 folds),0.980 +/- 0.007 (in 3 folds),0.813,0.769,0.029091,550,16,✅ no
elasticnet_cv0.75,0.981 +/- 0.006 (in 3 folds),0.981 +/- 0.006 (in 3 folds),0.827,0.784,0.029091,550,16,✅ no
lasso_cv,0.976 +/- 0.009 (in 3 folds),0.974 +/- 0.011 (in 3 folds),0.813,0.766,0.029091,550,16,✅ no
rf_multiclass,0.976 +/- 0.008 (in 3 folds),0.975 +/- 0.008 (in 3 folds),0.833,0.787,0.029091,550,16,✅ no
logisticregression_unregularized,0.970 +/- 0.007 (in 3 folds),0.968 +/- 0.009 (in 3 folds),0.805,0.755,0.029091,550,16,✅ no
linearsvm_ovr,0.965 +/- 0.007 (in 3 folds),0.964 +/- 0.010 (in 3 folds),0.787,0.734,0.029091,550,16,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.353,0.024,0.029091,550,16,❗ Missing classes!
dummy_stratified,0.494 +/- 0.022 (in 3 folds),0.505 +/- 0.010 (in 3 folds),0.200,-0.018,0.029091,550,16,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease, metamodel flavor isotype_counts_only

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.683 +/- 0.008 (in 3 folds),0.672 +/- 0.007 (in 3 folds),0.373,0.165,0.0,550,0,✅ no
rf_multiclass,0.679 +/- 0.022 (in 3 folds),0.655 +/- 0.014 (in 3 folds),0.395,0.210,0.0,550,0,✅ no
logisticregression_unregularized,0.671 +/- 0.021 (in 3 folds),0.664 +/- 0.014 (in 3 folds),0.367,0.201,0.0,550,0,✅ no
ridge_cv,0.664 +/- 0.017 (in 3 folds),0.661 +/- 0.013 (in 3 folds),0.362,0.189,0.0,550,0,✅ no
elasticnet_cv0.25,0.651 +/- 0.041 (in 3 folds),0.642 +/- 0.043 (in 3 folds),0.375,0.205,0.0,550,0,✅ no
elasticnet_cv,0.646 +/- 0.049 (in 3 folds),0.641 +/- 0.045 (in 3 folds),0.373,0.204,0.0,550,0,✅ no
elasticnet_cv0.75,0.637 +/- 0.065 (in 3 folds),0.643 +/- 0.041 (in 3 folds),0.378,0.207,0.0,550,0,✅ no
lasso_cv,0.635 +/- 0.068 (in 3 folds),0.641 +/- 0.044 (in 3 folds),0.371,0.197,0.0,550,0,✅ no
dummy_stratified,0.502 +/- 0.013 (in 3 folds),0.511 +/- 0.007 (in 3 folds),0.218,-0.001,0.0,550,0,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.358,0.000,0.0,550,0,❗ Missing classes!


In [9]:
# Demographic controlled
for target in [
    TargetObsColumnEnum.disease_all_demographics_present,
]:
    for gene_locus in config.gene_loci_used:
        choose(gene_locus, [target])
    choose(config.gene_loci_used, [target])

# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.969 +/- 0.003 (in 3 folds),0.969 +/- 0.003 (in 3 folds),0.845,0.777,0.011905,420,5,✅ no
ridge_cv,0.968 +/- 0.006 (in 3 folds),0.969 +/- 0.007 (in 3 folds),0.840,0.772,0.011905,420,5,✅ no
elasticnet_cv0.25,0.965 +/- 0.009 (in 3 folds),0.966 +/- 0.009 (in 3 folds),0.843,0.775,0.011905,420,5,✅ no
rf_multiclass,0.964 +/- 0.009 (in 3 folds),0.965 +/- 0.008 (in 3 folds),0.838,0.763,0.011905,420,5,✅ no
elasticnet_cv,0.963 +/- 0.012 (in 3 folds),0.965 +/- 0.011 (in 3 folds),0.838,0.768,0.011905,420,5,✅ no
elasticnet_cv0.75,0.962 +/- 0.013 (in 3 folds),0.964 +/- 0.012 (in 3 folds),0.845,0.778,0.011905,420,5,✅ no
lasso_cv,0.957 +/- 0.009 (in 3 folds),0.958 +/- 0.010 (in 3 folds),0.829,0.753,0.011905,420,5,✅ no
logisticregression_unregularized,0.956 +/- 0.011 (in 3 folds),0.956 +/- 0.012 (in 3 folds),0.805,0.724,0.011905,420,5,✅ no
dummy_stratified,0.504 +/- 0.004 (in 3 folds),0.506 +/- 0.001 (in 3 folds),0.333,0.014,0.011905,420,5,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.450,0.005,0.011905,420,5,❗ Missing classes!


# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor with_demographics_columns

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.968 +/- 0.007 (in 3 folds),0.967 +/- 0.007 (in 3 folds),0.838,0.763,0.011905,420,5,✅ no
elasticnet_cv0.25,0.962 +/- 0.002 (in 3 folds),0.961 +/- 0.004 (in 3 folds),0.829,0.756,0.011905,420,5,✅ no
elasticnet_cv,0.959 +/- 0.004 (in 3 folds),0.957 +/- 0.005 (in 3 folds),0.829,0.755,0.011905,420,5,✅ no
ridge_cv,0.957 +/- 0.016 (in 3 folds),0.958 +/- 0.013 (in 3 folds),0.824,0.754,0.011905,420,5,✅ no
lasso_cv,0.955 +/- 0.011 (in 3 folds),0.954 +/- 0.010 (in 3 folds),0.831,0.759,0.011905,420,5,✅ no
elasticnet_cv0.75,0.950 +/- 0.011 (in 3 folds),0.950 +/- 0.010 (in 3 folds),0.824,0.746,0.011905,420,5,✅ no
logisticregression_unregularized,0.945 +/- 0.014 (in 3 folds),0.940 +/- 0.011 (in 3 folds),0.812,0.730,0.011905,420,5,✅ no
linearsvm_ovr,0.933 +/- 0.020 (in 3 folds),0.935 +/- 0.014 (in 3 folds),0.814,0.732,0.011905,420,5,✅ no
dummy_stratified,0.504 +/- 0.004 (in 3 folds),0.506 +/- 0.001 (in 3 folds),0.333,0.014,0.011905,420,5,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.450,0.005,0.011905,420,5,❗ Missing classes!


# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor demographics_regressed_out

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv0.75,0.683 +/- 0.061 (in 3 folds),0.696 +/- 0.052 (in 3 folds),0.282,0.152,0.0,358,0,✅ no
lasso_cv,0.682 +/- 0.060 (in 3 folds),0.700 +/- 0.055 (in 3 folds),0.285,0.151,0.0,358,0,✅ no
logisticregression_unregularized,0.682 +/- 0.054 (in 3 folds),0.703 +/- 0.054 (in 3 folds),0.296,0.156,0.0,358,0,✅ no
elasticnet_cv,0.679 +/- 0.064 (in 3 folds),0.693 +/- 0.063 (in 3 folds),0.282,0.153,0.0,358,0,✅ no
rf_multiclass,0.669 +/- 0.032 (in 3 folds),0.652 +/- 0.036 (in 3 folds),0.441,0.210,0.0,358,0,✅ no
elasticnet_cv0.25,0.665 +/- 0.037 (in 3 folds),0.677 +/- 0.025 (in 3 folds),0.257,0.130,0.0,358,0,✅ no
ridge_cv,0.656 +/- 0.032 (in 3 folds),0.675 +/- 0.031 (in 3 folds),0.249,0.127,0.0,358,0,✅ no
linearsvm_ovr,0.638 +/- 0.042 (in 3 folds),0.659 +/- 0.045 (in 3 folds),0.416,0.121,0.0,358,0,❗ Missing classes!
dummy_stratified,0.502 +/- 0.006 (in 3 folds),0.505 +/- 0.003 (in 3 folds),0.330,-0.000,0.0,358,0,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.461,0.000,0.0,358,0,❗ Missing classes!


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor demographics_only_sex

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.588 +/- 0.028 (in 3 folds),0.551 +/- 0.019 (in 3 folds),0.324,0.120,0.0,358,0,❗ Missing classes!
logisticregression_unregularized,0.588 +/- 0.028 (in 3 folds),0.551 +/- 0.019 (in 3 folds),0.282,0.121,0.0,358,0,❗ Missing classes!
linearsvm_ovr,0.588 +/- 0.028 (in 3 folds),0.551 +/- 0.019 (in 3 folds),0.391,0.169,0.0,358,0,❗ Missing classes!
ridge_cv,0.588 +/- 0.028 (in 3 folds),0.551 +/- 0.019 (in 3 folds),0.282,0.121,0.0,358,0,❗ Missing classes!
lasso_cv,0.578 +/- 0.025 (in 3 folds),0.548 +/- 0.016 (in 3 folds),0.288,0.102,0.0,358,0,✅ no
elasticnet_cv,0.578 +/- 0.025 (in 3 folds),0.548 +/- 0.016 (in 3 folds),0.324,0.120,0.0,358,0,❗ Missing classes!
elasticnet_cv0.25,0.578 +/- 0.025 (in 3 folds),0.548 +/- 0.016 (in 3 folds),0.282,0.121,0.0,358,0,❗ Missing classes!
elasticnet_cv0.75,0.559 +/- 0.027 (in 3 folds),0.540 +/- 0.017 (in 3 folds),0.324,0.120,0.0,358,0,❗ Missing classes!
dummy_stratified,0.502 +/- 0.006 (in 3 folds),0.505 +/- 0.003 (in 3 folds),0.330,-0.000,0.0,358,0,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.461,0.000,0.0,358,0,❗ Missing classes!


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor demographics_only_ethnicity_condensed

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.793 +/- 0.028 (in 3 folds),0.769 +/- 0.028 (in 3 folds),0.654,0.497,0.0,358,0,✅ no
ridge_cv,0.792 +/- 0.018 (in 3 folds),0.766 +/- 0.016 (in 3 folds),0.620,0.472,0.0,358,0,✅ no
elasticnet_cv0.25,0.790 +/- 0.023 (in 3 folds),0.763 +/- 0.023 (in 3 folds),0.550,0.416,0.0,358,0,✅ no
rf_multiclass,0.788 +/- 0.021 (in 3 folds),0.760 +/- 0.021 (in 3 folds),0.620,0.472,0.0,358,0,✅ no
elasticnet_cv0.75,0.785 +/- 0.013 (in 3 folds),0.762 +/- 0.021 (in 3 folds),0.550,0.416,0.0,358,0,✅ no
lasso_cv,0.785 +/- 0.013 (in 3 folds),0.762 +/- 0.021 (in 3 folds),0.550,0.416,0.0,358,0,✅ no
elasticnet_cv,0.784 +/- 0.012 (in 3 folds),0.761 +/- 0.020 (in 3 folds),0.550,0.416,0.0,358,0,✅ no
logisticregression_unregularized,0.767 +/- 0.005 (in 3 folds),0.751 +/- 0.004 (in 3 folds),0.620,0.472,0.0,358,0,✅ no
dummy_stratified,0.502 +/- 0.006 (in 3 folds),0.505 +/- 0.003 (in 3 folds),0.330,-0.000,0.0,358,0,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.461,0.000,0.0,358,0,❗ Missing classes!


In [10]:
# Demographics from healthy
for target in [
    TargetObsColumnEnum.ethnicity_condensed_healthy_only,
    TargetObsColumnEnum.age_group_healthy_only,
    TargetObsColumnEnum.age_group_binary_healthy_only,
    TargetObsColumnEnum.age_group_pediatric_healthy_only,
    TargetObsColumnEnum.sex_healthy_only,
]:
    for gene_locus in config.gene_loci_used:
        choose(gene_locus, [target])
    choose(config.gene_loci_used, [target])

# GeneLocus.BCR, TargetObsColumnEnum.ethnicity_condensed_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.781 +/- 0.071 (in 3 folds),0.780 +/- 0.079 (in 3 folds),0.681,0.441,0.073298,191,14,❗ Missing classes!
ridge_cv,0.755 +/- 0.084 (in 3 folds),0.752 +/- 0.065 (in 3 folds),0.550,0.337,0.073298,191,14,✅ no
elasticnet_cv0.25,0.734 +/- 0.047 (in 3 folds),0.729 +/- 0.043 (in 3 folds),0.565,0.308,0.073298,191,14,✅ no
elasticnet_cv,0.734 +/- 0.042 (in 3 folds),0.730 +/- 0.034 (in 3 folds),0.555,0.297,0.073298,191,14,✅ no
elasticnet_cv0.75,0.731 +/- 0.044 (in 3 folds),0.727 +/- 0.036 (in 3 folds),0.555,0.297,0.073298,191,14,✅ no
linearsvm_ovr,0.725 +/- 0.064 (in 3 folds),0.708 +/- 0.048 (in 3 folds),0.565,0.315,0.073298,191,14,✅ no
lasso_cv,0.725 +/- 0.043 (in 3 folds),0.717 +/- 0.035 (in 3 folds),0.539,0.282,0.073298,191,14,✅ no
logisticregression_unregularized,0.722 +/- 0.061 (in 3 folds),0.706 +/- 0.044 (in 3 folds),0.513,0.253,0.073298,191,14,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.539,0.037,0.073298,191,14,❗ Missing classes!
dummy_stratified,0.483 +/- 0.036 (in 3 folds),0.498 +/- 0.008 (in 3 folds),0.382,-0.022,0.073298,191,14,✅ no


# GeneLocus.TCR, TargetObsColumnEnum.ethnicity_condensed_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
ridge_cv,0.713 +/- 0.024 (in 3 folds),0.730 +/- 0.038 (in 3 folds),0.624,0.395,0.036364,165,6,✅ no
elasticnet_cv0.25,0.712 +/- 0.036 (in 3 folds),0.733 +/- 0.036 (in 3 folds),0.491,0.295,0.036364,165,6,✅ no
elasticnet_cv,0.709 +/- 0.029 (in 3 folds),0.733 +/- 0.033 (in 3 folds),0.636,0.378,0.036364,165,6,✅ no
logisticregression_unregularized,0.705 +/- 0.012 (in 3 folds),0.714 +/- 0.029 (in 3 folds),0.624,0.349,0.036364,165,6,✅ no
elasticnet_cv0.75,0.701 +/- 0.030 (in 3 folds),0.722 +/- 0.039 (in 3 folds),0.576,0.260,0.036364,165,6,✅ no
linearsvm_ovr,0.700 +/- 0.050 (in 3 folds),0.705 +/- 0.053 (in 3 folds),0.570,0.252,0.036364,165,6,✅ no
rf_multiclass,0.696 +/- 0.036 (in 3 folds),0.723 +/- 0.037 (in 3 folds),0.673,0.403,0.036364,165,6,✅ no
lasso_cv,0.668 +/- 0.026 (in 3 folds),0.705 +/- 0.026 (in 3 folds),0.564,0.273,0.036364,165,6,✅ no
dummy_stratified,0.526 +/- 0.051 (in 3 folds),0.514 +/- 0.016 (in 3 folds),0.455,0.048,0.036364,165,6,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.564,-0.014,0.036364,165,6,❗ Missing classes!


# GeneLocus.BCR|TCR, TargetObsColumnEnum.ethnicity_condensed_healthy_only, metamodel flavor default

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.529 +/- 0.170 (in 2 folds),0.590 +/- 0.161 (in 2 folds),0.373,-0.006,0.269841,126,34,✅ no
elasticnet_cv,0.517 +/- 0.024 (in 2 folds),0.526 +/- 0.018 (in 2 folds),0.373,-0.016,0.269841,126,34,✅ no
dummy_stratified,0.511 +/- 0.066 (in 2 folds),0.520 +/- 0.003 (in 2 folds),0.389,0.031,0.269841,126,34,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 2 folds),0.512 +/- 0.038 (in 2 folds),0.381,-0.003,0.269841,126,34,✅ no
ridge_cv,0.475 +/- 0.095 (in 2 folds),0.508 +/- 0.010 (in 2 folds),0.405,0.062,0.269841,126,34,✅ no
logisticregression_unregularized,0.472 +/- 0.092 (in 2 folds),0.508 +/- 0.010 (in 2 folds),0.389,0.031,0.269841,126,34,✅ no
linearsvm_ovr,0.472 +/- 0.091 (in 2 folds),0.508 +/- 0.010 (in 2 folds),0.389,0.031,0.269841,126,34,✅ no
elasticnet_cv0.75,0.471 +/- 0.089 (in 2 folds),0.506 +/- 0.010 (in 2 folds),0.389,0.031,0.269841,126,34,✅ no
lasso_cv,0.470 +/- 0.088 (in 2 folds),0.506 +/- 0.010 (in 2 folds),0.389,0.031,0.269841,126,34,✅ no
elasticnet_cv0.25,0.455 +/- 0.051 (in 2 folds),0.511 +/- 0.005 (in 2 folds),0.365,-0.012,0.269841,126,34,✅ no


# GeneLocus.TCR, TargetObsColumnEnum.sex_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
dummy_stratified,0.513 +/- 0.032 (in 3 folds),0.570 +/- 0.075 (in 3 folds),0.455,0.048,0.151515,165,25,✅ no
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.563 +/- 0.087 (in 3 folds),0.442,0.006,0.151515,165,25,✅ no
rf_multiclass,0.471 +/- 0.107 (in 3 folds),0.587 +/- 0.093 (in 3 folds),0.424,-0.001,0.151515,165,25,✅ no
ridge_cv,0.455 +/- 0.077 (in 3 folds),0.537 +/- 0.092 (in 3 folds),0.388,-0.031,0.151515,165,25,✅ no
logisticregression_unregularized,0.393 +/- 0.023 (in 3 folds),0.517 +/- 0.106 (in 3 folds),0.406,-0.033,0.151515,165,25,✅ no
linearsvm_ovr,0.392 +/- 0.024 (in 3 folds),0.517 +/- 0.106 (in 3 folds),0.406,-0.033,0.151515,165,25,✅ no
elasticnet_cv0.25,0.392 +/- 0.024 (in 3 folds),0.516 +/- 0.106 (in 3 folds),0.406,-0.033,0.151515,165,25,✅ no
elasticnet_cv0.75,0.391 +/- 0.026 (in 3 folds),0.524 +/- 0.105 (in 3 folds),0.400,-0.045,0.151515,165,25,✅ no
elasticnet_cv,0.391 +/- 0.026 (in 3 folds),0.516 +/- 0.106 (in 3 folds),0.400,-0.045,0.151515,165,25,✅ no
lasso_cv,0.389 +/- 0.029 (in 3 folds),0.523 +/- 0.105 (in 3 folds),0.376,-0.085,0.151515,165,25,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.sex_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
dummy_most_frequent,0.500 +/- 0.000 (in 2 folds),0.559 +/- 0.029 (in 2 folds),0.273,-0.035,0.418182,110,46,✅ no
dummy_stratified,0.489 +/- 0.008 (in 2 folds),0.553 +/- 0.025 (in 2 folds),0.291,0.003,0.418182,110,46,✅ no
rf_multiclass,0.475 +/- 0.140 (in 2 folds),0.606 +/- 0.130 (in 2 folds),0.300,0.014,0.418182,110,46,✅ no
elasticnet_cv0.75,0.462 +/- 0.054 (in 2 folds),0.544 +/- 0.008 (in 2 folds),0.255,-0.073,0.418182,110,46,✅ no
elasticnet_cv,0.460 +/- 0.056 (in 2 folds),0.543 +/- 0.007 (in 2 folds),0.255,-0.073,0.418182,110,46,✅ no
lasso_cv,0.459 +/- 0.058 (in 2 folds),0.542 +/- 0.006 (in 2 folds),0.245,-0.091,0.418182,110,46,✅ no
ridge_cv,0.457 +/- 0.060 (in 2 folds),0.531 +/- 0.011 (in 2 folds),0.264,-0.056,0.418182,110,46,✅ no
logisticregression_unregularized,0.399 +/- 0.034 (in 2 folds),0.522 +/- 0.040 (in 2 folds),0.264,-0.051,0.418182,110,46,✅ no
linearsvm_ovr,0.399 +/- 0.026 (in 2 folds),0.522 +/- 0.036 (in 2 folds),0.264,-0.051,0.418182,110,46,✅ no
elasticnet_cv0.25,0.396 +/- 0.026 (in 2 folds),0.509 +/- 0.020 (in 2 folds),0.245,-0.084,0.418182,110,46,✅ no
